In [21]:
import os
import numpy as np
import pandas as pd
import argparse
import pickle
import tqdm
import json
from sklearn import mixture, linear_model, svm, gaussian_process
import h5py

In [22]:
def get_stratified_blocked_train_test_idxs(df, sample_id_label, class_label, test_size=0.20, random_state=0):
    np.random.seed(random_state)
    
    class_mean = df.groupby([sample_id_label]).ClinVar_labels.mean()
    class_quantiles = pd.qcut(dataset.groupby('protein_name').ClinVar_labels.mean(), 10)
    
    quantile_df = {}
    for gi, quant in enumerate(set(class_quantiles.values.to_list())):
        quantile_df[quant] = gi
    class_quantiles = class_quantiles.replace(quantile_df)
    
    train_sample_ids = []
    test_sample_ids = []
    for q in set(class_quantiles.values):
        b = class_quantiles[class_quantiles==q].copy().index.values
        test_sample_ids.extend(b[:round(len(b)*test_size)])
        train_sample_ids.extend(b[round(len(b)*test_size):])
    
    train_idxs = np.where(np.in1d(df[sample_id_label], np.array(train_sample_ids)))[0]   
    test_idxs = np.where(np.in1d(df[sample_id_label], np.array(test_sample_ids)))[0] 
                                
    return train_idxs, test_idxs

In [31]:
data_dir = '/home/scratch/wpotosna/disease_variant_prediction_language_model/data'
logodds_dir = '/home/scratch/wpotosna/data'
gmm_dir = '/home/scratch/wpotosna/gmm'

mapping_file = pd.read_csv(data_dir+'/labels/ClinVar_labels_P53_PTEN_RASH_SCN5A.csv',low_memory=False)
protein_list = [i.replace('.txt', '') for i in os.listdir(data_dir+'/wildtype_protein_sequences/') if (i != 'ACHA_HUMAN')& (i!='.ipynb_checkpoints')]
list_variables_to_keep=['protein_name','mutations','evol_indices']
protein_GMM_weight = 0.3


labels = pd.read_csv(data_dir+'/labels/PTEN_ClinVar_labels.csv')
logodds = pd.read_csv(logodds_dir+'/single_logodds.csv', sep=';')

protein_names = ['_'.join(i.split('_')[:-1]) for i in logodds.SeqID_SAV.values]
mutation_names = [i.split('_')[-1] for i in logodds.SeqID_SAV.values]

df = pd.DataFrame(data=[protein_names, mutation_names, logodds.SAV_score.values]).T
df.columns = ['protein_name', 'mutations', 'SAV_score']
df.set_index(['protein_name', 'mutations'], inplace=True)

clinvar_labels = labels.set_index(['protein_name', 'mutations']).loc[df.index.values]

dataset = pd.concat([df, clinvar_labels], axis=1)
dataset.reset_index(inplace=True, drop=False)

dataset.drop(index=dataset[dataset.protein_name=='ALAT2_HUMAN'].index.values[0], inplace=True)


train_idxs, test_idxs = get_stratified_blocked_train_test_idxs(df=dataset, 
                                                               sample_id_label='protein_name', 
                                                               class_label='ClinVar_labels', 
                                                               test_size=0.20, 
                                                               random_state=0)
# EVE computes negative log ratio: -(log(Xv) - log(Xw))
dataset.SAV_score = dataset.SAV_score*(-1)
X_train = dataset.iloc[train_idxs]
X_test = dataset.iloc[test_idxs]

In [32]:
dict_models = {}
dict_pathogenic_cluster_index = {}

main_GMM = mixture.GaussianMixture(n_components=2, covariance_type='full',max_iter=1000,n_init=30,tol=1e-4)
main_GMM.fit(X_train.SAV_score.values.reshape(-1, 1))
        
dict_models['main'] = main_GMM
pathogenic_cluster_index = np.argmax(np.array(main_GMM.means_).flatten()) #The pathogenic cluster is the cluster with higher mean value
dict_pathogenic_cluster_index['main'] = pathogenic_cluster_index

with open(gmm_dir+'/gmm_stats.csv', "a") as logs:
    logs.write(",".join(str(x) for x in [
        'main', np.array(main_GMM.weights_).flatten()[dict_pathogenic_cluster_index['main']], np.array(main_GMM.means_).flatten()[dict_pathogenic_cluster_index['main']],
        np.array(main_GMM.means_).flatten()[1 - dict_pathogenic_cluster_index['main']], np.sqrt(np.array(main_GMM.covariances_).flatten()[dict_pathogenic_cluster_index['main']]),
        np.sqrt(np.array(main_GMM.covariances_).flatten()[1 - dict_pathogenic_cluster_index['main']])])+"\n")

for protein in tqdm.tqdm(dataset.protein_name.unique(), "Training all protein GMMs"):
    X_train_protein = dataset[dataset.protein_name==protein].SAV_score.values.reshape(-1, 1)
    if len(X_train_protein) > 0: #We have evol indices computed for protein on file
        protein_GMM = mixture.GaussianMixture(n_components=2,covariance_type='full',max_iter=1000,tol=1e-4,weights_init=main_GMM.weights_,means_init=main_GMM.means_,precisions_init=main_GMM.precisions_)
        protein_GMM.fit(X_train_protein)
        dict_models[protein] = protein_GMM
        dict_pathogenic_cluster_index[protein] = np.argmax(np.array(protein_GMM.means_).flatten())
        with open(gmm_dir+'/gmm_stats.csv', "a") as logs:
            logs.write(",".join(str(x) for x in [
                protein, np.array(protein_GMM.weights_).flatten()[dict_pathogenic_cluster_index[protein]], np.array(protein_GMM.means_).flatten()[dict_pathogenic_cluster_index[protein]],
                np.array(protein_GMM.means_).flatten()[1 - dict_pathogenic_cluster_index[protein]], np.sqrt(np.array(protein_GMM.covariances_).flatten()[dict_pathogenic_cluster_index[protein]]),
                np.sqrt(np.array(protein_GMM.covariances_).flatten()[1 - dict_pathogenic_cluster_index[protein]])
                ])+"\n")

pickle.dump(dict_models, open(gmm_dir+'/GMM_model_dictionary.pkl', 'wb'))
pickle.dump(dict_pathogenic_cluster_index, open(gmm_dir+'/GMM_pathogenic_cluster_index_dictionary.pkl', 'wb'))


Training all protein GMMs: 100%|█████████████████████████████████████████████████████████| 103/103 [00:00<00:00, 118.38it/s]


# Compute EVE scores


In [33]:


def compute_weighted_score_two_GMMs(X_pred, main_model, protein_model, cluster_index_main, cluster_index_protein, protein_weight):
    return protein_model.predict_proba(X_pred)[:,cluster_index_protein] * protein_weight + (main_model.predict_proba(X_pred)[:,cluster_index_main]) * (1 - protein_weight)

def compute_weighted_class_two_GMMs(X_pred, main_model, protein_model, cluster_index_main, cluster_index_protein, protein_weight):
    """By construct, 1 is always index of pathogenic, 0 always that of benign"""
    proba_pathogenic = protein_model.predict_proba(X_pred)[:,cluster_index_protein] * protein_weight + (main_model.predict_proba(X_pred)[:,cluster_index_main]) * (1 - protein_weight)
    return (proba_pathogenic > 0.5).astype(int)

def compute_EVE_scores(X_test, dict_models, dict_pathogenic_cluster_index, protein_GMM_weight):
    all_scores = X_test.copy()
    if protein_GMM_weight > 0.0:
        for protein in tqdm.tqdm(X_test.protein_name.unique(),"Scoring all protein mutations"):
            
            X_test_protein = X_test[X_test.protein_name==protein].SAV_score.values.reshape(-1, 1)
            mutation_scores_protein = compute_weighted_score_two_GMMs(X_pred=X_test_protein, 
                                                                            main_model = dict_models['main'], 
                                                                            protein_model=dict_models[protein], 
                                                                            cluster_index_main = dict_pathogenic_cluster_index['main'], 
                                                                            cluster_index_protein = dict_pathogenic_cluster_index[protein], 
                                                                            protein_weight = protein_GMM_weight)
            gmm_class_protein = compute_weighted_class_two_GMMs(X_pred=X_test_protein, 
                                                                            main_model = dict_models['main'], 
                                                                            protein_model=dict_models[protein], 
                                                                            cluster_index_main = dict_pathogenic_cluster_index['main'], 
                                                                            cluster_index_protein = dict_pathogenic_cluster_index[protein], 
                                                                            protein_weight = protein_GMM_weight)

            gmm_class_label_protein = pd.Series(gmm_class_protein).map(lambda x: 'Pathogenic' if x == 1 else 'Benign')
                    
            all_scores.loc[all_scores.protein_name==protein, 'EVE_scores'] = np.array(mutation_scores_protein)
            all_scores.loc[all_scores.protein_name==protein, 'EVE_classes_100_pct_retained'] = np.array(gmm_class_label_protein)
            all_scores.loc[all_scores.EVE_classes_100_pct_retained=='Benign', 'EVE_classes_100_pct_retained'] = 0.0
            all_scores.loc[all_scores.EVE_classes_100_pct_retained=='Pathogenic', 'EVE_classes_100_pct_retained'] = 1.0

    else:
        all_scores = all_evol_indices.copy()
        mutation_scores = dict_models['main'].predict_proba(np.array(all_scores['evol_indices']).reshape(-1, 1))
        all_scores['EVE_scores'] = mutation_scores[:,dict_pathogenic_cluster_index['main']]
        gmm_class = dict_models['main'].predict(np.array(all_scores['evol_indices']).reshape(-1, 1))
        all_scores['EVE_classes_100_pct_retained'] = np.array(pd.Series(gmm_class).map(lambda x: 'Pathogenic' if x == dict_pathogenic_cluster_index['main'] else 'Benign'))
        
    return all_scores

In [34]:
test_results = compute_EVE_scores(X_test, dict_models, dict_pathogenic_cluster_index, protein_GMM_weight)
test_results.to_csv(logodds_dir+'/test_results.csv', index=False)

Scoring all protein mutations: 100%|███████████████████████████████████████████████████████| 20/20 [00:00<00:00, 367.23it/s]


In [35]:
train_results = compute_EVE_scores(X_train, dict_models, dict_pathogenic_cluster_index, protein_GMM_weight)

Scoring all protein mutations: 100%|███████████████████████████████████████████████████████| 83/83 [00:00<00:00, 316.09it/s]


In [36]:
train_results.to_csv(logodds_dir+'/train_results.csv', index=False)